# Tensor Decomposition Analysis with `trokachat_tensor_decomp`

This notebook demonstrates how to use the `trokachat_tensor_decomp` package to analyze tensor data from two experimental conditions, estimate the optimal tensor rank, and perform tensor decomposition. It replicates the analysis from the original script but uses the new modular package structure.

## 1. Import Required Modules

First, let's import the necessary modules from our package:

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import functions from our package
from trokachat_tensor_decomp import (
    load_tensor_data,
    create_sparse_tensor,
    calc_denoising_loss_parallel,
    plot_mse_boxplot,
    perform_decomposition,
    save_decomposition_results
)

# Define a helper function for saving dimension mappings since it's not in the package yet
def save_dimension_mappings(dim_mappings, output_dir):
    """Save dimension mappings to CSV files"""
    os.makedirs(output_dir, exist_ok=True)
    mapping_files = []
    
    for i, mapping in enumerate(dim_mappings):
        mapping_df = pd.DataFrame(list(mapping.items()), columns=[f'Category_{i}', 'Index'])
        mapping_file_path = os.path.join(output_dir, f"mapping_{i}.csv")
        mapping_df.to_csv(mapping_file_path, index=False)
        mapping_files.append(mapping_file_path)
        
    return mapping_files

## 2. Load and Process Tensor Data

Let's load the tensor data from the CSV files using the exact same file paths from the original script.

In [ ]:
# Define file paths from the original script
tensor_data_path = "/Users/troks27/Desktop/DIAB_NG_TrokaChatML/TrokaChat/2 Condition Tensor/2condition_tensor_data.csv"
tensor_dimensions_path = "/Users/troks27/Desktop/DIAB_NG_TrokaChatML/TrokaChat/2 Condition Tensor/2condition_tensor_dimensions.csv"

# Define output directories
output_dir = "/Users/troks27/Desktop/DIAB_NG_TrokaChatML/TrokaChat/Decomposed Tensor/"
rank_results_dir = "/Users/troks27/Desktop/DIAB_NG_TrokaChatML/TrokaChat/Tensor Rank Determination/"

# Create directories if they don't exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(rank_results_dir, exist_ok=True)

# Load tensor data using our package function
tensor_data, tensor_dimensions = load_tensor_data(tensor_data_path, tensor_dimensions_path)

# Display information about the data
print(f"Loaded tensor data with {len(tensor_data)} rows")
print("Columns in tensor data:")
print(tensor_data.columns)
tensor_data.head()

## 3. Create Sparse Tensor

Now, we'll convert the DataFrame to a sparse tensor representation using our package function.

In [ ]:
# Define dimension columns (same as in the original script)
dimension_columns = ['Source', 'Target', 'Ligand_Receptor', 'Condition']

# Create sparse tensor
sparse_tl_tensor, tensor_shape, dim_mappings = create_sparse_tensor(tensor_data, dimension_columns)

# Display information about the tensor
print(f"Created sparse tensor with shape: {tensor_shape}")
print(f"Number of non-zero elements: {sparse_tl_tensor.nnz}")
sparse_tl_tensor

## 4. Estimate Tensor Rank

We'll determine the optimal tensor rank through denoising loss calculation for different noise levels. This replicates the analysis from the original script.

In [ ]:
# Use the same parameters as in the original script
rank_max = 30
noise_levels = [0.05, 0.1, 0.2, 0.3, 0.4]
n_rep = 20

# We'll just run one noise level for demonstration
# In practice, you'd run all noise levels as in the original script
sample_noise = 0.1  # Choose a sample noise level for demonstration

print(f"Estimating tensor rank with noise level {sample_noise}...")
avg_errors, err_summary_df = calc_denoising_loss_parallel(
    sparse_tl_tensor, 
    rank_max=rank_max, 
    noise=sample_noise, 
    n_rep=n_rep,
    method="MSE"
)

# Save the results
csv_filename = f"{rank_results_dir}denoising_mask_noise={sample_noise}_rank_estim.csv"
err_summary_df.to_csv(csv_filename, index=False)
print(f"Saved rank estimation results to {csv_filename}")

## 5. Visualize Rank Estimation Results

Let's visualize the rank estimation results using our package's visualization function.

In [ ]:
# Create plot for the sample noise level
output_filename = f"{rank_results_dir}denoising_mask_noise={sample_noise}_rank_estim_plot.pdf"
plot_mse_boxplot(csv_filename=csv_filename, output_filename=output_filename)

# Plot the average errors
plt.figure(figsize=(10, 6))
plt.plot(range(1, rank_max + 1), avg_errors, marker='o')
plt.yscale('log')
plt.xlabel("Rank")
plt.ylabel("Loss")
plt.title(f"Average Loss vs Rank (Noise = {sample_noise})")
plt.grid(True)
plt.show()

# Find the optimal rank
optimal_rank = np.argmin(avg_errors) + 1
print(f"Optimal rank based on minimum loss: {optimal_rank}")

## 6. Process All Noise Levels

In a full analysis, you would process all noise levels and create plots for each one. Here's how you would do that:

In [ ]:
# This cell shows how to process all noise levels
# We're commenting it out to avoid long computation

'''
for noise in noise_levels:
    print(f"\nProcessing noise level: {noise}")
    
    # Run the denoising loss calculation
    avg_errors, err_summary_df = calc_denoising_loss_parallel(
        sparse_tl_tensor, 
        rank_max=rank_max, 
        noise=noise, 
        n_rep=n_rep
    )
    
    # Save the DataFrame to a CSV file
    csv_filename = f"{rank_results_dir}denoising_mask_noise={noise}_rank_estim.csv"
    err_summary_df.to_csv(csv_filename, index=False)
    
    # Create and save the plot
    output_filename = f"{rank_results_dir}denoising_mask_noise={noise}_rank_estim_plot.pdf"
    plot_mse_boxplot(csv_filename=csv_filename, output_filename=output_filename)
'''

## 7. Load and Visualize Existing Results

If you've already run the analysis and saved the results, you can load and visualize them:

In [ ]:
# Visualize all noise levels from previously saved results
for noise in noise_levels:
    csv_filename = f'{rank_results_dir}denoising_mask_noise={noise}_rank_estim.csv'
    if os.path.exists(csv_filename):
        output_filename = f'{rank_results_dir}denoising_mask_noise={noise}_rank_estim_plot.pdf'
        plot_mse_boxplot(csv_filename=csv_filename, output_filename=output_filename)
        print(f"Created plot for noise level {noise}")
    else:
        print(f"File not found: {csv_filename}")

## 8. Perform Tensor Decomposition

Based on the rank estimation, we'll perform tensor decomposition with the optimal rank. In the original script, rank 16 was chosen.

In [ ]:
# Set the rank for decomposition (same as in the original script)
rank = 16  # Based on rank estimation from the original analysis

# Perform non-negative CP decomposition using our package function
print(f"Performing tensor decomposition with rank {rank}...")
factors = perform_decomposition(
    sparse_tl_tensor, 
    rank=rank, 
    init='random', 
    verbose=True, 
    n_iter_max=1000
)

print("Tensor decomposition completed successfully!")

## 9. Save Decomposition Results

Now, we'll save the decomposition results (weights and factors) using our package function.

In [ ]:
# Save dimension mappings
mapping_files = save_dimension_mappings(dim_mappings, output_dir)
print(f"Saved dimension mappings to {output_dir}")

# Save decomposition results
saved_files = save_decomposition_results(factors, output_dir)

print(f"Weights saved to: {saved_files.get('weights')}")
print(f"Factor matrices saved to:")
for factor_file in saved_files.get('factors', []):
    print(f"  - {factor_file}")

## 10. Analyze Decomposition Results

Let's analyze the decomposition results to understand the patterns in the data.

In [ ]:
# Extract weights and factors
weights, factors_list = factors

# Convert weights to dense array
weights_array = np.array(weights.todense()).flatten()

# Plot weights
plt.figure(figsize=(10, 6))
plt.bar(range(1, len(weights_array) + 1), weights_array)
plt.xlabel('Component')
plt.ylabel('Weight')
plt.title('Component Weights')
plt.grid(True, alpha=0.3)
plt.show()

# Analyze the factor matrices
dimension_names = ['Source', 'Target', 'Ligand_Receptor', 'Condition']

for i, factor in enumerate(factors_list):
    print(f"\nAnalyzing Factor {i} ({dimension_names[i]})")
    print(f"Shape: {factor.shape}")
    
    # Convert to dense if needed
    if hasattr(factor, 'todense'):
        factor_dense = factor.todense()
    else:
        factor_dense = factor
    
    # Plot heatmap for this factor
    plt.figure(figsize=(12, 8))
    sns.heatmap(factor_dense, cmap='viridis')
    plt.xlabel('Component')
    plt.ylabel(f'{dimension_names[i]} Index')
    plt.title(f'Factor Matrix for {dimension_names[i]}')
    plt.tight_layout()
    plt.show()
    
    # For the condition factor (which is small), display the values
    if i == 3:  # Condition factor
        factor_df = pd.DataFrame(factor_dense, columns=[f'Component_{j+1}' for j in range(rank)])
        # Map indices to actual condition names
        reverse_mapping = {v: k for k, v in dim_mappings[i].items()}
        factor_df.index = [reverse_mapping.get(idx, idx) for idx in range(len(factor_df))]
        print("Condition factor values:")
        display(factor_df)

## 11. Examine Component Contributions

Let's identify the top contributors for each component across the different dimensions.

In [ ]:
# Function to get top contributors for a factor and component
def get_top_contributors(factor_matrix, component_idx, dimension_mapping, top_n=10):
    # Convert to dense if needed
    if hasattr(factor_matrix, 'todense'):
        factor_dense = factor_matrix.todense()
    else:
        factor_dense = factor_matrix
    
    # Get the values for this component
    component_values = factor_dense[:, component_idx]
    
    # Create a DataFrame with indices and values
    df = pd.DataFrame({
        'Index': range(len(component_values)),
        'Value': component_values
    })
    
    # Sort by value in descending order
    df = df.sort_values('Value', ascending=False)
    
    # Get the top N contributors
    top_contributors = df.head(top_n)
    
    # Map indices to actual names
    reverse_mapping = {v: k for k, v in dimension_mapping.items()}
    top_contributors['Name'] = top_contributors['Index'].map(reverse_mapping)
    
    return top_contributors[['Name', 'Value']]

# Analyze a specific component (e.g., component 0)
component_to_analyze = 0

print(f"\nAnalyzing Component {component_to_analyze + 1}")
print("------------------------------")

for i, factor in enumerate(factors_list):
    print(f"\nTop contributors in {dimension_names[i]}:")
    top_contributors = get_top_contributors(factor, component_to_analyze, dim_mappings[i], top_n=5)
    display(top_contributors)

## Conclusion

In this notebook, we've demonstrated how to use the `trokachat_tensor_decomp` package to:

1. Load tensor data from CSV files
2. Create a sparse tensor representation
3. Estimate the optimal tensor rank using denoising loss calculation
4. Visualize the rank estimation results
5. Perform tensor decomposition with the selected rank
6. Save and analyze the decomposition results

The package provides a clean, modular interface to tensor decomposition operations, making the analysis workflow more readable and maintainable.